In [73]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from typing import TypedDict
from dotenv import load_dotenv
import json
import glob


In [74]:
load_dotenv()

True

In [75]:
json_files = sorted(glob.glob("data/*.json"))[:3]

json_data = [json.load(open(f, encoding="utf-8")) for f in json_files]


In [79]:
import os
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY") 
)

In [67]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    json_data: list

In [68]:
def blog_title(state: BlogState) -> BlogState:
    # Convert the JSONs into trimmed readable chunks
    context = "\n\n".join(
        [f"JSON {i+1}:\n{json.dumps(j, indent=2)[:1500]}..." for i, j in enumerate(state["json_data"])]
    )

    # Create system + human prompt
    messages = [
        SystemMessage(
            content="You are a top-tier content strategist and expert blog writer. Your job is to create compelling, hook-worthy titles that instantly grab attention."
        ),
        HumanMessage(
            content=f"""Based on the following JSON context, generate an engaging and clickable blog title:

{context}

Guidelines:
- Make it short, attention-grabbing and relevant to the data.
- Avoid clickbait. Prefer clarity + curiosity.
- Output ONLY the title — no explanations or prefix.

Now generate the blog title:"""
        )
    ]

    # Call the model
    result = model.invoke(messages)

    # ✅ Fix: Store the generated title into the state
    title = result.content.strip()
    state["title"] = title

    # Optional: Print to confirm
    print("\n🔥 Generated Blog Title:\n", title)

    return state


In [69]:
from langchain_core.messages import SystemMessage, HumanMessage

def blog_outline(state: BlogState) -> BlogState:
    title = state.get("title", "")

    # Dynamically form context from JSONs
    context = "\n\n".join(
        [f"JSON {i+1}:\n{json.dumps(j, indent=2)[:1500]}..." for i, j in enumerate(state["json_data"])]
    )

    # Define system + human prompt
    messages = [
        SystemMessage(content="You are an expert blog strategist. Your task is to generate a structured and insightful outline for a blog based on provided context."),
        HumanMessage(content=f"""
Below is data extracted from multiple JSON files. Based on the context and the blog title provided, generate a clear, structured outline for the blog.

📌 Title: {title}

📂 Context:
{context}

✅ Instructions:
- Make the outline informative and logically ordered
- Include at least 4–6 sections
- Output ONLY the outline (no intro, no comments)

Now generate the blog outline below:
        """)
    ]

    # Call the model
    result = model.invoke(messages)

    # Update and return state
    state["outline"] = result.content.strip()
    print("\n📌 Generated Blog Outline:\n", state["outline"])
    return state


In [70]:
def blog_content(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]

    context = "\n\n".join(
        [f"JSON {i+1}:\n{json.dumps(j, indent=2)[:2000]}..." for i, j in enumerate(state["json_data"])]
    )

    prompt = ChatPromptTemplate.from_template(
        """You are generating a blog article.

Context from the source files:
{context}

Here is the blog title: {title}

Here is the outline to follow:
{outline}

Now write the full blog article.
"""
    )

    chain = prompt | model

    result = chain.invoke({
        "context": context,
        "title": title,
        "outline": outline
    })

    state["content"] = result.content
    return state

In [71]:
graph = StateGraph(BlogState)
graph.add_node("blog_title", blog_title)
graph.add_node("blog_outline", blog_outline)
graph.add_node("blog_content", blog_content)

graph.add_edge(START, "blog_title")
graph.add_edge("blog_title", "blog_outline")
graph.add_edge("blog_outline", "blog_content")
graph.add_edge("blog_content", END)

workflow = graph.compile()



In [72]:
# Run the workflow
initial_state = {
    'json_data': json_data
}

final_state = workflow.invoke(initial_state)

# Print the final generated blog content
print(final_state["title"])
print(final_state["outline"])
print(final_state["content"])

# Save to file
with open("final_output.md", "w", encoding="utf-8") as f:
    f.write(final_state["content"])


🔥 Generated Blog Title:
 "Why Responsible AI Must Lead the Charge Before Regulations Catch Up"

📌 Generated Blog Outline:
 I. Introduction
   - Brief overview of the rapid advancement of AI technologies
   - Importance of responsible AI governance ahead of regulations

II. The Current Landscape of AI Regulation
   - Overview of existing AI regulations like GDPR and the European AI Act
   - Challenges posed by the rapid pace of AI development outpacing regulatory frameworks

III. The Imperative for Proactive AI Governance
   - Explanation of why waiting for regulations is insufficient
   - Importance of bridging theoretical ethics with practical governance
   - The role of infrastructure in translating values into action

IV. Building Trust in AI Systems
   - The fragility of trust in AI systems
   - Risks of data breaches and incidents harming brand reputation
   - Strategies for maintaining trust through responsible AI practices

V. A Three-Layer Governance Model
   - Cultural Layer: